## Setup

In [1]:
import getpass
import os

#os.environ["OPENAI_API_KEY"] = getpass.getpass()
#os.environ["SLACK_USER_TOKEN"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = ""
os.environ["SLACK_USER_TOKEN"] = ""
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
#document loading
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, Tool, AgentType, create_json_chat_agent
from langchain.agents.react.agent import create_react_agent
from langchain import hub

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.agent_toolkits import SlackToolkit

## Loading and splitting Documents

In [3]:
loader = PyPDFLoader("handbook.pdf")
pages = loader.load()
print('pages:',len(pages))

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)
print('split docs:',len(docs))

pages: 46
split docs: 177


In [4]:
!rm -rf ./docs/chroma 

## Embedding 

In [5]:
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

177


## Retriever

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
question = "What is the name of the company?"
docs = vectordb.similarity_search(question,k=3)
#print(len(docs))
qa_chain.invoke({"query": question})

{'query': 'What is the name of the company?',
 'result': 'The name of the company is Zania, Inc.'}

## Creating Tools for Agent

In [7]:
toolkit = SlackToolkit()
tool = toolkit.get_tools()
tool1 = tool[3].as_tool(name = 'slack', description='send message on slack')
tool2 = create_retriever_tool(retriever=vectordb.as_retriever(), name = 'Retrival_tool', description='Retrival_tool_for_documentdb')
tools = [tool2, tool1]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input":" Answer the question What is the name of the company? and send answer to slack channel #testproject"})



> Entering new AgentExecutor chain...

Invoking: `Retrival_tool` with `{'query': 'What is the name of the company?'}`


This policy may not be appropriate in its entirety for employees working in Montana.
2.0 
Introductory Language and Policies
2.1 
About the Company
[[Add your about the company statement here.]]
2.2 
Company Facilities
[[Insert information about your company facilities here.]]
2.3 
Ethics Code
Zania, Inc. will conduct business honestly and ethically wherever operations are maintained. We strive to improve the quality
of our services, products, and operations and will maintain a reputation for honesty, fairness, respect, responsibility,
integrity, trust, and sound business judgment. Our managers and employees are expected to adhere to high standards of
business and personal integrity as a representation of our business practices, at all times consistent with their duty of loyalty
to the Company.
We expect that officers, directors, and employees will not knowingly mis

{'input': ' Answer the question What is the name of the company? and send answer to slack channel #testproject',
 'output': 'The name of the company is Zania, Inc. I have sent this information to the Slack channel #testproject.'}